# LinkedIn Data Cleaning ("clean_jobs.csv")

This notebook performs the data cleaning and preprocessing steps to transform raw LinkedIn job data i.e. data/bronze/`clean_jobs.csv` which is ingested via `src/data_ingestion.py` on monthly basis using GitHub Actions. We'll use the outcomes to improve our `etl.py` which will processed our raw data into the "silver" layer (enriched_jobs.csv).

## 1. Setup and Imports

Jupyter Lab does **not** automatically add our project root to `sys.path`, so it cannot find the src package by default.

### **How to Fix**

Add the following code cell **at the top of your notebook (before your imports):**

````python
import sys
from pathlib import Path

# Add project root to sys.path so 'src' can be imported
sys.path.append(str(Path().resolve().parent))
````

**Then your imports will work:**
```python
from src.extractors.salary_extractor import SalaryETL
from src.extractors.experience_extractor import categorize_experience
from src.extractors.skills_extractor import extract_skills
from src.extractors.job_type_extractor import extract_work_type, extract_employment_type
```

---

**Summary:**  
Add the project root to `sys.path` at the top of your notebook to fix `ModuleNotFoundError: No module named 'src'`.

In [19]:
# Import necessary libraries
import sys
from pathlib import Path
## Add project root to sys.path so 'src' can be imported
sys.path.append(str(Path().resolve().parent))
## EDA libraries
import pandas as pd
import numpy as np
from pathlib import Path

# custom extractors and processors
from src.extractors.salary_extractor import SalaryETL
from src.extractors.experience_extractor import categorize_experience
from src.extractors.skills_extractor import extract_skills
from src.extractors.job_type_extractor import extract_work_type, extract_employment_type

 ## 2. Load Raw Data (Bronze)

In [2]:
BRONZE_PATH = Path("../data/bronze/clean_jobs.csv")
df_raw = pd.read_csv(BRONZE_PATH)
print("Raw data shape:", df_raw.shape)
df_raw.head()

Raw data shape: (618, 10)


,id,title,company,location,link,source,date_posted,work_type,employment_type,description
0,1,Data Analyst,Meta,"New York, NY",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,2025-04-14,NaN,NaN,The Social Measurement team is a growing team ...
1,2,Data Analyst,Meta,"San Francisco, CA",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,2025-04-14,NaN,NaN,The Social Measurement team is a growing team ...
2,3,Data Analyst,Meta,"Los Angeles, CA",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,2025-04-14,NaN,NaN,The Social Measurement team is a growing team ...
3,4,Data Analyst,Meta,"Washington, DC",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,2025-04-14,NaN,NaN,The Social Measurement team is a growing team ...
4,5,Data Analyst II,Pinterest,"Chicago, IL",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,2025-04-16,NaN,NaN,About Pinterest\r\n\r\nMillions of people arou...


### 2.1 Inspect Raw Columns and Values

- List columns, data types
- Count missingness per column

In [3]:
print(df_raw.info())
print(df_raw.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 618 entries, 0 to 617
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               618 non-null    int64  
 1   title            618 non-null    object 
 2   company          618 non-null    object 
 3   location         618 non-null    object 
 4   link             618 non-null    object 
 5   source           618 non-null    object 
 6   date_posted      618 non-null    object 
 7   work_type        0 non-null      float64
 8   employment_type  0 non-null      float64
 9   description      614 non-null    object 
dtypes: float64(2), int64(1), object(7)
memory usage: 48.4+ KB
None
id                   0
title                0
company              0
location             0
link                 0
source               0
date_posted          0
work_type          618
employment_type    618
description          4
dtype: int64


## 3. Cleaning and Preprocessing

### 3.1 Standardize Text Fields

- Strip whitespace, lowercase titles and descriptions

In [4]:
df = df_raw
df['title'] = df['title'].str.strip()
df['description'] = df['description'].fillna('').str.strip()

### 3.2 Extract Features

#### 3.2.1 Salary Extraction

In [5]:
salary_etl = SalaryETL()
df = salary_etl.process_job_dataframe(
    df,
    text_column='description',
    include_title=True,
    title_column='title'
)

[Warning] Pattern 4 failed to parse: not enough values to unpack (expected 4, got 3)
[Warning] Pattern 4 failed to parse: not enough values to unpack (expected 4, got 3)
[Warning] Pattern 4 failed to parse: not enough values to unpack (expected 4, got 3)
[Warning] Pattern 4 failed to parse: not enough values to unpack (expected 4, got 3)
[Warning] Pattern 4 failed to parse: not enough values to unpack (expected 4, got 3)
[Warning] Pattern 4 failed to parse: not enough values to unpack (expected 4, got 3)
[Warning] Pattern 4 failed to parse: not enough values to unpack (expected 4, got 3)
[Warning] Pattern 4 failed to parse: not enough values to unpack (expected 4, got 3)
[Warning] Pattern 4 failed to parse: not enough values to unpack (expected 4, got 3)
[Warning] Pattern 4 failed to parse: not enough values to unpack (expected 4, got 3)
[Warning] Pattern 4 failed to parse: not enough values to unpack (expected 4, got 3)
[Warning] Pattern 4 failed to parse: not enough values to unpack 

#### 3.2.2 Experience Level

In [6]:
df['experience_level'] = df.apply(
    lambda r: categorize_experience(r['title'], r['description']),
    axis=1
)

#### 3.2.3 Skill Extraction

In [7]:
skills = df['description'].apply(extract_skills)
for col in ['programming_languages','libraries','analyst_tools','cloud_platforms']:
    df[col] = skills.apply(lambda x: x.get(col, []))

#### 3.2.4 Job & Employment Type

In [8]:
df['work_type'] = df.apply(
    lambda r: extract_work_type(r['title'], r['description']), axis=1
)
df['employment_type'] = df.apply(
    lambda r: extract_employment_type(r['title'], r['description']), axis=1
)

### 3.3 Handle Missing and Invalid Data

- Fill or flag missing salary fields (has_salary)
- Flag extreme salary outliers
- Normalize salary periods and currency

**Note:** The fallback to geopy/countryinfo for currency should only be used if no valid salary and currency were extracted from the job text. If a salary and currency are present in the description/title, those should always be used and not overwritten by location-based inference.

In [9]:
df.head()

,id,title,company,location,link,source,date_posted,work_type,employment_type,description,...,salary_period,min_salary_annual_usd,max_salary_annual_usd,avg_salary_annual_usd,salary_confidence,experience_level,programming_languages,libraries,analyst_tools,cloud_platforms
0,1,Data Analyst,Meta,"New York, NY",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,2025-04-14,Not Specified,Not Specified,The Social Measurement team is a growing team ...,...,None,124000.0,124000.0,124000.0,0.6,Not Specified,"[python, r, sql]",[],[tableau],[]
1,2,Data Analyst,Meta,"San Francisco, CA",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,2025-04-14,Not Specified,Not Specified,The Social Measurement team is a growing team ...,...,None,124000.0,124000.0,124000.0,0.6,Not Specified,"[python, r, sql]",[],[tableau],[]
2,3,Data Analyst,Meta,"Los Angeles, CA",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,2025-04-14,Not Specified,Not Specified,The Social Measurement team is a growing team ...,...,None,124000.0,124000.0,124000.0,0.6,Not Specified,"[python, r, sql]",[],[tableau],[]
3,4,Data Analyst,Meta,"Washington, DC",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,2025-04-14,Not Specified,Not Specified,The Social Measurement team is a growing team ...,...,None,124000.0,124000.0,124000.0,0.6,Not Specified,"[python, r, sql]",[],[tableau],[]
4,5,Data Analyst II,Pinterest,"Chicago, IL",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,2025-04-16,Not Specified,Not Specified,About Pinterest\r\n\r\nMillions of people arou...,...,None,None,None,None,None,Not Specified,"[python, sql]",[],[],[]


In [10]:
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 618 entries, 0 to 617
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     618 non-null    int64 
 1   title                  618 non-null    object
 2   company                618 non-null    object
 3   location               618 non-null    object
 4   link                   618 non-null    object
 5   source                 618 non-null    object
 6   date_posted            618 non-null    object
 7   work_type              618 non-null    object
 8   employment_type        618 non-null    object
 9   description            618 non-null    object
 10  has_salary             618 non-null    object
 11  currency               618 non-null    object
 12  min_salary_raw         149 non-null    object
 13  max_salary_raw         149 non-null    object
 14  single_salary_raw      146 non-null    object
 15  salary_period          

id                         0
title                      0
company                    0
location                   0
link                       0
source                     0
date_posted                0
work_type                  0
employment_type            0
description                0
has_salary                 0
currency                   0
min_salary_raw           469
max_salary_raw           469
single_salary_raw        472
salary_period            583
min_salary_annual_usd    323
max_salary_annual_usd    323
avg_salary_annual_usd    323
salary_confidence        323
experience_level           0
programming_languages      0
libraries                  0
analyst_tools              0
cloud_platforms            0
dtype: int64

In [11]:
unique_currencies = df['currency'].unique()
print(unique_currencies)

['USD' 'ZAR' 'GBP' 'TOP' 'EUR']


In [12]:
df.to_csv('output.csv', index=False)

In [13]:
max_salary = df['avg_salary_annual_usd'].max()
print(f"Maximum average salary: {max_salary}")

Maximum average salary: 240000.0


# LinkedIn Data Cleaning ("enriched_jobs.csv")¶

Final check before EDA

## Load Silver Data `enriched_jobs.csv`

In [2]:
SILVER_PATH = Path("../data/silver/enriched_jobs.csv")
df_silver = pd.read_csv(SILVER_PATH)
print("Silver data shape:", df_silver.shape)
df_silver.head()

Silver data shape: (1048, 26)


,id,title,company,location,link,source,date_posted,work_type,employment_type,description,...,salary_period,min_salary_annual_usd,max_salary_annual_usd,avg_salary_annual_usd,salary_confidence,experience_level,programming_languages,libraries,analyst_tools,cloud_platforms
0,1,Data Analyst,Meta,"New York, NY",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,2025-04-14,Not Specified,Full-time,The Social Measurement team is a growing team ...,...,NaN,124000.0,124000.0,124000.0,0.6,Not Specified,"['python', 'r', 'sql']",[],['tableau'],[]
1,2,Data Analyst,Meta,"San Francisco, CA",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,2025-04-14,Not Specified,Full-time,The Social Measurement team is a growing team ...,...,NaN,124000.0,124000.0,124000.0,0.6,Not Specified,"['python', 'r', 'sql']",[],['tableau'],[]
2,3,Data Analyst,Meta,"Los Angeles, CA",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,2025-04-14,Not Specified,Full-time,The Social Measurement team is a growing team ...,...,NaN,124000.0,124000.0,124000.0,0.6,Not Specified,"['python', 'r', 'sql']",[],['tableau'],[]
3,4,Data Analyst,Meta,"Washington, DC",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,2025-04-14,Not Specified,Full-time,The Social Measurement team is a growing team ...,...,NaN,124000.0,124000.0,124000.0,0.6,Not Specified,"['python', 'r', 'sql']",[],['tableau'],[]
4,5,Data Analyst II,Pinterest,"Chicago, IL",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,2025-04-16,Hybrid,Full-time,About Pinterest\r\n\r\nMillions of people arou...,...,NaN,NaN,NaN,NaN,NaN,Not Specified,"['python', 'sql']",[],[],[]


### Inspect Raw Columns and Values

- List columns, data types
- Count missingness per column

In [38]:
print(df_silver.info())
print(df_silver.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048 entries, 0 to 1047
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     1048 non-null   int64  
 1   title                  1048 non-null   object 
 2   company                1048 non-null   object 
 3   location               1048 non-null   object 
 4   link                   1048 non-null   object 
 5   source                 959 non-null    object 
 6   date_posted            1025 non-null   object 
 7   work_type              1048 non-null   object 
 8   employment_type        1048 non-null   object 
 9   description            1048 non-null   object 
 10  header_text            971 non-null    object 
 11  has_salary             1048 non-null   bool   
 12  currency_raw           1048 non-null   object 
 13  min_salary_raw         91 non-null     float64
 14  max_salary_raw         92 non-null     float64
 15  sing

In [9]:
df_silver[df_silver['company'].isnull()]

,id,title,company,location,link,source,date_posted,work_type,employment_type,description,...,salary_period,min_salary_annual_usd,max_salary_annual_usd,avg_salary_annual_usd,salary_confidence,experience_level,programming_languages,libraries,analyst_tools,cloud_platforms
947,2344,Business Analyst (BA),NaN,"Remote, US",https://www.indeed.com/viewjob?jk=1247796689ad...,NaN,2025-06-10,Remote,Full-time,MoCaFi is committed to building a financial se...,...,NaN,90000.0,90000.0,90000.0,0.6,Not Specified,[],[],"['jira', 'visio']",[]
1001,2441,Healthcare Data Engineer (Remote - Virtual Dem...,NaN,"Remote, US",https://www.indeed.com/viewjob?jk=86f58c4e9351...,indeed,2025-06-06,Remote,Not Specified,**If you got into healthcare to make a differe...,...,NaN,NaN,NaN,NaN,NaN,Not Specified,"['python', 'sql']",['spark'],"['azure', 'postgresql', 'snowflake']","['aws', 'azure', 'bigquery', 'gcp', 'redshift'..."


In [10]:
# populate missing company names
df_silver.loc[df_silver['id'] == 2344, 'company'] = "MoCaFi (Mobility Capital Finance, Inc)"
df_silver.loc[df_silver['id'] == 2441, 'company'] = "Rippl Care"

In [15]:
# Find missing descriptions
df_silver[df_silver['description'].isnull()]

,id,title,company,location,link,source,date_posted,work_type,employment_type,description,...,salary_period,min_salary_annual_usd,max_salary_annual_usd,avg_salary_annual_usd,salary_confidence,experience_level,programming_languages,libraries,analyst_tools,cloud_platforms
359,876,Data Fulfillment Analyst,National Basketball Association (NBA),"New York, NY",https://www.linkedin.com/jobs/view/data-fulfil...,LinkedIn,2025-05-18,On-site,Full-time,NaN,...,NaN,100000.0,100000.0,100000.0,0.6,Not Specified,[],[],[],[]
361,878,Data Analyst - People Analytics (Short-Term Em...,Meta,"New York, NY",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,2025-05-21,Not Specified,Full-time,NaN,...,NaN,124000.0,124000.0,124000.0,0.6,Not Specified,[],[],[],[]
364,881,"Specialist, Commercial Data Reporting and Anal...",Delta Air Lines,"Atlanta, GA",https://www.linkedin.com/jobs/view/specialist-...,LinkedIn,2025-05-15,On-site,Full-time,NaN,...,NaN,NaN,NaN,NaN,NaN,Mid-Level,[],[],[],[]
415,939,Data Analyst I,Booking.com,"Amsterdam, North Holland, Netherlands",https://nl.linkedin.com/jobs/view/data-analyst...,LinkedIn,2025-05-21,Hybrid,Full-time,NaN,...,NaN,NaN,NaN,NaN,NaN,Not Specified,[],[],[],[]


In [18]:
# Populate missing descriptions & Skills
## ID = 876
df_silver.loc[df_silver['id'] == 876, 'description'] = """
About the job
WORK OPTION: The NBA currently provides eligible employees the option of working remotely one day per week.

At the NBA, we're passionate about growing and celebrating the game of basketball. Through the intensity of the game and the amazing athletic skill of our players, we deliver excitement to hundreds of millions of fans around the world.

As a global sports and media business, the NBA is so much more. While Basketball Operations runs the league's on-court activities, other departments manage relationships with television and digital media partners, develop marketing partnerships with some of the world's most recognizable companies, oversee the licensing of NBA merchandise, and handle a wide range of responsibilities that drive the NBA's success.

Essential Functions And Job Summary

The Data Fulfillment Analyst will be responsible for helping to facilitate the use of the NBA’s vast fan data, data strategy, and digital marketing and technology capabilities to enable our partners to use these assets to grow their business. The set of partners will vary greatly and will include those from the retail, strategic, content and marketing partnership spaces creating an opportunity to work with a diverse mix of brands and business. The ideal candidate is customer and project focused and enjoys the challenges of working across functions with team members from Data Strategy, Global Partnerships & Media, Direct-To-Consumer, Legal and Data Engineering. You will draw on your experience across digital marketing, data analysis and AdTech/MarTech to solve complex problems that drive value to both the NBA and its partners.

Major Responsibilities

 Apply subject matter expertise to develop and deliver digital marketing strategies and programs that accelerate customer acquisition, retention and brand awareness for partner data collaboration projects
 Consult with the Global Partnerships & Media team and with peers on the Technology & Platform Strategy team on the right mix of tactics to deliver the partners’ objectives (pre-sales)
 Lead the execution of tactics defined by the Global Partnership & Media team for each partner (post-sales)
 Lead the implementation of data cleanroom solutions including setup and configuration
 Provide support and guidance to internal and external stakeholders on data requirements and optimal implementation
 Collaborate with partners on data overlap analysis, segmentation and audience development
 Ensure marketable audiences are successfully onboarded to ad platforms (e.g., Meta, Google, etc.) for ad targeting
 Make optimization recommendations based on partner objectives and performance
 Manage project from start to finish to ensure that all facets of the partner agreement have been successfully followed
 Responsible for keeping a pulse on trends in marketing/ad technology and marketing operations

Required Education/Professional Experience

 2+ years of experience working in AdTech, product operations, product management, or a technical marketing operations capacity
 Bachelor’s degree required

Required Skills/Knowledge Attributes

 Experience with CRM platform technology and data structures
 Demonstrated SQL skills required
 Proficiency with evaluating data in Microsoft Excel required with abilities to leverage statistical programming software (e.g., R) to manipulated larger data sets [preferred / a plus]
 Knowledge of clean room technology and data onboarding tools
 Understanding of how to use common tools to conduct measurement of marketing performance and how to communicate results to stakeholders with varying levels of expertise in data
 Expert at problem-solving and critical thinking
 Ability to successfully navigate cross-functional teams and prioritize across competing needs
 Strong written and verbal communication skills

Salary Range: $100,000 - $115,000

We Consider Applicants For All Positions On The Basis Of Merit, Qualifications And Business Needs, And Without Regard To Race, Color, National Origin, Religion, Sex, Gender Identity, Age, Disability, Alienage Or Citizenship Status, Ancestry, Marital Status, Creed, Genetic Predisposition Or Carrier Status, Sexual Orientation, Veteran Status, Familial Status, Status As A Victim Of Domestic Violence Or Any Other Status Or Characteristic Protected By Applicable Federal, State, Or Local Law.

The NBA is committed to providing a safe and healthy workplace. To safeguard our employees and their families, our visitors, and the broader community from COVID-19, and in consideration of recommendations from health authorities and the NBA’s own advisors, any individual working onsite in our New York and New Jersey offices must be fully vaccinated against COVID-19. The NBA will discuss accommodations for individuals who cannot be vaccinated due to a medical reason or sincerely held religious belief, practice, or observance.
"""

# The extracted information
programming_languages_list = ['SQL', 'R']
libraries_list = []
analyst_tools_list = [
    'Microsoft Excel', 
    'CRM platform technology', 
    'Clean room technology', 
    'Data onboarding tools', 
    'Meta (ad platform)', 
    'Google (ad platform)'
]
cloud_platforms_list = []

# Find the specific row index for id 876
# .index[0] assumes the 'id' is unique and gets the first (and only) index label
row_index = df_silver[df_silver['id'] == 876].index[0]

# Use .at for efficient single-cell assignment
# This correctly assigns the entire list as a single object to the cell
df_silver.at[row_index, 'programming_languages'] = programming_languages_list
df_silver.at[row_index, 'libraries'] = libraries_list
df_silver.at[row_index, 'analyst_tools'] = analyst_tools_list
df_silver.at[row_index, 'cloud_platforms'] = cloud_platforms_list

In [20]:
# Populate missing descriptions & Skills
## ID = 878
df_silver.loc[df_silver['id'] == 878, 'description'] = """
About the job
Data Analysts in People Analytics drive the business by empowering HR and business leaders to incorporate people insights into workforce and business decisions. In this role, you will create analysis with people data, create dashboards for scaled solutions and efficient reporting, and develop materials for the executive leaders of the company to drive strategy. You will partner with People Analytics teams to influence priorities, perform research and build solutions for the future. The ideal candidate will have demonstrated analytical and technical skills, experience creating data-driven executive level presentations and will thrive managing concurrent projects while working with a cross-functional team. We are looking for someone who enjoys building, figuring out the best way to achieve results, and working closely with teams across Meta to solve complex problems.

Data Analyst - People Analytics (Short-Term Employment) Responsibilities:

Apply expertise in data analytics to identify business relevant insights and develop executive level presentations to influence people strategy
Act as an internal consultant and partner to HR and business leaders and engage with the broader analytics community to deliver insights, research and scalable solutions
Focus on impact and propose metrics to understand success of People initiatives
Manage multiple concurrent projects that require inputs from cross-functional stakeholders, while balancing impact on business needs
Deliver analytical findings to executive audiences
Leverage data analysis tools, such as Python, R, SQL, Tableau, and spreadsheets to develop models, understand trends, and extract insights
Demonstrate exceptional judgment and discretion when dealing with highly sensitive people data

Minimum Qualifications:

BA/BS in Engineering, Industrial Psychology, Data Science, or related field with 5+ years (or MBA/MA/MS with 4+ years or PhD and 3+ years) experience in fields such as analytics, management consulting, sales operations, finance, compensation, or related areas
5+ years of data analysis, quantitative / qualitative research, experience gathering and evaluating data and validating hypotheses
3+ years of SQL, data modeling and advanced querying
3+ years of program management, directing tasks and managing multiple streams of work
Experience engaging directly with executive stakeholders and managing relationships
3+ years of experience in data visualization tools (for example, Tableau) and spreadsheets (complex formulas, pivots, macros, etc.)
3+ years of storytelling with data, communicating technical results to technical and non-technical audiences

Preferred Qualifications:

Experience working with HR/organizational people data (e.g. headcount, turnover, recruiting metrics, and other people analytics)
Experience working across various levels of an organization, while building trust and fostering collaboration
2+ years of dashboard development using advanced Tableau functionality (e.g. data blending)
5+ years of experience SQL and data pipeline development and automation
3+ years of experience with Python or R and developing solutions that scale

About Meta:

Meta builds technologies that help people connect, find communities, and grow businesses. When Facebook launched in 2004, it changed the way people connect. Apps like Messenger, Instagram and WhatsApp further empowered billions around the world. Now, Meta is moving beyond 2D screens toward immersive experiences like augmented and virtual reality to help build the next evolution in social technology. People who choose to build their careers by building with us at Meta help shape a future that will take us beyond what digital connection makes possible today—beyond the constraints of screens, the limits of distance, and even the rules of physics.

Meta is proud to be an Equal Employment Opportunity and Affirmative Action employer. We do not discriminate based upon race, religion, color, national origin, sex (including pregnancy, childbirth, or related medical conditions), sexual orientation, gender, gender identity, gender expression, transgender status, sexual stereotypes, age, status as a protected veteran, status as an individual with a disability, or other applicable legally protected characteristics. We also consider qualified applicants with criminal histories, consistent with applicable federal, state and local law. Meta participates in the E-Verify program in certain locations, as required by law. Please note that Meta may leverage artificial intelligence and machine learning technologies in connection with applications for employment.

Meta is committed to providing reasonable accommodations for candidates with disabilities in our recruiting process. If you need any assistance or accommodations due to a disability, please let us know at accommodations-ext@fb.com.

$124,000/year to $176,000/year + benefits

Individual compensation is determined by skills, qualifications, experience, and location. Compensation details listed in this posting reflect the base hourly rate, monthly rate, or annual salary only, and do not include bonus, equity or sales incentives, if applicable. In addition to base compensation, Meta offers benefits. Learn more about  benefits  at Meta.
"""
# The extracted information for ID = 878
programming_languages_list = ['Python', 'R', 'SQL']
libraries_list = []
analyst_tools_list = ['Tableau', 'Spreadsheets']
cloud_platforms_list = []

# Find the specific row index for id 878
# .index[0] assumes the 'id' is unique
row_index_878 = df_silver[df_silver['id'] == 878].index[0]

# Use .at for efficient single-cell assignment
df_silver.at[row_index_878, 'programming_languages'] = programming_languages_list
df_silver.at[row_index_878, 'libraries'] = libraries_list
df_silver.at[row_index_878, 'analyst_tools'] = analyst_tools_list
df_silver.at[row_index_878, 'cloud_platforms'] = cloud_platforms_list

In [22]:
# Populate missing descriptions & Skills
## ID = 881
df_silver.loc[df_silver['id'] == 881, 'description'] = """
About the job
United States, Georgia, Atlanta

Enterprise Digital Strategy

 13-Mar-2025

Ref #: 27814

How you'll help us Keep Climbing (overview & key responsibilities)

Enterprise Solutions (ES) is seeking a Specialist, Commercial Data Reporting & Analysis to lead the development and ongoing support of varied data sources and decision support tools to help advance Delta's commercial strategy. Delta’s commercial strategy is centered on Delta’s ability to expand adoption of premium products, grow customer trust and be a world-class retailer all while maintaining our industry leading unit revenue performance.

The successful candidate will play a critical role in delivering key data sources needed to support Commercial and broader Enterprise decision making, where applicable. This individual will be able to partner across business and IT teams to create and deliver valuable data, training, and/or tools across the Delta Enterprise. This role will conduct data analysis as needed to improve data quality or answer various business questions as well as develop and/or enhance commercial reports as needed. The selected candidate will combine strong business acumen and technical skills to drive data enablement and decision making across the organization. We are looking for candidates who have a passion for data, are motivated by solving problems, exhibit curiosity to understand current processes, and can think creatively to identify solutions.

Responsibilities

Work with business and IT partners to translate business requirements into usable and reliable data solutions and drive implementation of data solutions
Independently lead the design, modeling and creation of new commercial data sources
Identify and understand Delta commercial data opportunities and priorities to deliver timely and reliable solutions that meet business needs
Perform ad hoc analysis on commercial metrics and data sets as requested
Act as a subject matter expert for multiple commercial data sets and reports
Effectively communicate complex technical subject matter to a non-technical audience as well as create/deliver training for commercial data and tools

Benefits and Perks to Help You Keep Climbing

Benefits

Our culture is rooted in a shared dedication to living our values – Care, Integrity, Resilience and Servant Leadership – every day, in everything we do. At Delta, our people are our success. At the heart of what we offer is our focus on Sharing Success with Delta employees. Exploring a career at Delta gives you a chance to see the world while earning great compensation and benefits to help you keep climbing along the way:

Competitive salary, industry-leading proﬁt sharing program, and performance incentives. 
401(k) with generous company contributions up to 9%. 
New hires are eligible for up to 2-weeks of vacation. This is earned for use in the following vacation year (April 1 – March 31). 
In addition to vacation, new hires are eligible for up to 56 hours of paid personal time within a 12-month period. 
10 paid holidays per calendar year. 
Birthing parents are eligible for 12-weeks of paid maternity/parental leave. 
Non-birthing parents are eligible for 2-weeks of paid parental leave. 
Comprehensive health beneﬁts including medical, dental, vision, short/long term disability and life insurance beneﬁts. 
Family care assistance through fertility support, surrogacy and adoption assistance, lactation support, subsidized back-up care, and programs that help with loved ones in all stages. 
Holistic Wellbeing programs to support physical, emotional, social, and financial health, including access to an employee assistance program offering support for you and anyone in your household, free financial coaching, and extensive resources supporting mental health. 
Domestic and International space-available flight privileges for employees and eligible family members. 
Career development programs to achieve your long-term career goals. 
World-wide partnerships to engage in community service and innovative goals created to focus on sustainability and reducing our carbon footprint. 
Business Resource Groups created to connect employees with common interests to promote inclusion, provide perspective and help implement strategies. 
Recognition rewards and awards through the platform Unstoppable Together. 
Access to over 500 discounts, specialty savings and voluntary benefits through Deltaperks such as car and hotel rentals and auto, home, and pet insurance, legal services, and childcare. 

What You Need To Succeed (minimum Qualifications)

 At least 3-5 years of work experience in a Data Engineering, Data Management, Data Analysis, Statistics role 
 Expertise in Python, R, SQL, Tableau, SAS, UNIX, Hyperion/BRIO, Excel, PowerBI or similar skills 
 Ability to understand large, unstructured data sets. 
 Understanding of data modeling and modern software architecture styles. 
 Demonstrates intellectual curiosity as well as sound analytical and communication skills; ability to communicate at all levels 
 Shown success in understanding and improving complex processes. 
 Capable of managing multiple projects with limited supervision in a fast-paced environment 
 Focused on results and can prioritize competing interests with seemingly disparate goals 
 Uses data to quantify problems, identify trade-offs and develop solutions 
 Consistently prioritizes safety and security of self, others, and personal data. 
 Embraces diverse people, thinking, and styles. 
 Possesses a high school diploma, GED, or high school equivalency. 
 Is at least 18 years of age and has authorization to work in the United States. 

What Will Give You a Competitive Edge (preferred Qualifications)

BA/BS or advanced degree in Engineering, Statistics, Data Analytics, Computer Science, Information Systems, Economics or similar
Experience in at least one commercial or operations function such as Reservations, Airport Customer Service, eCommerce, Revenue Management, Network Planning or Sales 
Knowledge of revenue management theory or airline systems/data 

"""
# The extracted information for ID = 881
programming_languages_list = ['Python', 'R', 'SQL', 'SAS']
libraries_list = []
analyst_tools_list = ['Tableau', 'UNIX', 'Hyperion/BRIO', 'Excel', 'PowerBI']
cloud_platforms_list = []

# Find the specific row index for id 881
# .index[0] assumes the 'id' is unique
row_index_881 = df_silver[df_silver['id'] == 881].index[0]

# Use .at for efficient single-cell assignment
df_silver.at[row_index_881, 'programming_languages'] = programming_languages_list
df_silver.at[row_index_881, 'libraries'] = libraries_list
df_silver.at[row_index_881, 'analyst_tools'] = analyst_tools_list
df_silver.at[row_index_881, 'cloud_platforms'] = cloud_platforms_list

In [23]:
# Populate missing descriptions & Skills
## ID = 939
df_silver.loc[df_silver['id'] == 939, 'description'] = """
About the job
Key Responsibilities

Responsible for owning the design and delivery of parts of Data Analytics solutions based on business requirements, contributing to bringing them from the initial idea-generation phase to implementation with some supervision.
Responsible for working independently on consuming data, preparing it for analytical use and translating business problems to data stories. Can ask for guidance on more complex tasks.
Responsible for working within the scope of their team and delivering work that informs business decisions for the product / business topic they currently work on.
Responsible for leveraging foundational skills in data analysis and data visualization to answer business questions. They incorporate stakeholder requirements when creating intuitive and insightful visualizations that translate data into actionable insights.
Responsible for connecting and collaborating with stakeholders in operational and project-based roles under the guidance of a manager. Uses effective communication to address their needs and presents data insights in an accessible, business-friendly manner.
Responsible for being flexible in adopting existing internal Data Analytics approaches and expanding their technical competencies when a more efficient way presents itself.
Responsible for growing their own craft skills while leveraging Data Analytics for impact on well defined business problems.
Responsible for supporting junior peers when opportunity presents itself through knowledge sharing and setting a good example.
Responsible for networking and proactively connecting with craft peers beyond the team's scope.
Must be familiar with the larger data ecosystem at Booking and how it relates to the work they do.
Responsible for connecting with peers in related crafts (ie. Data Engineering) as needed on specific projects to ensure successful outcomes.
Must have high level knowledge about the goals of their area (ie. track / vertical).
Responsible for ensuring quality of their own work by validating it through peer review.


Communication.Stakeholder

Business Stakeholders
Craft team
Managers
Leadership team
DSnA Community
Other tech crafts


Communication.Type

Cooperation, Information - Input, feedback and providing data outcomes to steer the business .
Cooperation - Support & collaboration
Cooperation, Information - Support / Input, feedback and providing data outcomes to steer the business .
Information - Providing data to steer the business.
Cooperation, Information - Knowledge sharing, collaboration and support
Information - Data Collection & tracking


Communication.Frequency

Frequent
Continuous
Frequent
Occasional
Frequent
Occasional


Level of Education.Level of Education

Bachelor degree
Master degree


Level Of Education.Description

Years of relevant Job Knowledge.Years of relevant Job Knowledge

Basic Job Knowledge (1 - 3 years)


Requirements Of Special Knowledge/skills

Driven and results oriented personality
Open to feedback and always looking for ways to improve own performance
Able to understand details while keeping an eye on the bigger picture
Analytical and curious mindset
Strong interpersonal skills
Open and friendly attitude
Strong knowledge and experience on data analysis.
Experience in data management, data visualization and quality.
Knowledge and experience using data analysis and visualization tools (e.g: tableau, data studio, powerbi, mixpanel, etc)
Knowledge and experience with at least one data analysis language (e.g: R, SQL, Python, etc)


Pre-Employment Screening

If your application is successful, your personal data may be used for a pre-employment screening check by a third party as permitted by applicable law. Depending on the vacancy and applicable law, a pre-employment screening may include employment history, education and other information (such as media information) that may be necessary for determining your qualifications and suitability for the position.
"""
# The extracted information for ID = 939
programming_languages_list = ['R', 'SQL', 'Python']
libraries_list = []
analyst_tools_list = ['Tableau', 'Data Studio', 'PowerBI', 'Mixpanel']
cloud_platforms_list = []

# Find the specific row index for id 939
# .index[0] assumes the 'id' is unique
row_index_939 = df_silver[df_silver['id'] == 939].index[0]

# Use .at for efficient single-cell assignment
df_silver.at[row_index_939, 'programming_languages'] = programming_languages_list
df_silver.at[row_index_939, 'libraries'] = libraries_list
df_silver.at[row_index_939, 'analyst_tools'] = analyst_tools_list
df_silver.at[row_index_939, 'cloud_platforms'] = cloud_platforms_list

In [26]:
# Find missing 15 locations
df_silver[df_silver['location'].isnull()]

,id,title,company,location,link,source,date_posted,work_type,employment_type,description,...,salary_period,min_salary_annual_usd,max_salary_annual_usd,avg_salary_annual_usd,salary_confidence,experience_level,programming_languages,libraries,analyst_tools,cloud_platforms
897,2279,"Sr. Analyst, Data & Analytics",Authentic Brands Group,NaN,https://www.linkedin.com/jobs/view/4229293070,NaN,2025-06-07,Not Specified,Full-time,**Who We Are** \r\n\r\n\r\n\r\n \r\n\r\n\r\n ...,...,NaN,105000.0,105000.0,105000.0,0.6,Senior,"['go', 'sql']",[],"['excel', 'looker']","['aws', 'bigquery']"
919,2304,Data Scientist,OGC Global,NaN,https://www.linkedin.com/jobs/view/4244768294,NaN,2025-06-08,Remote,Full-time,OGC is a boutique research and consulting agen...,...,NaN,NaN,NaN,NaN,NaN,Not Specified,"['python', 'r', 'sql', 'vba']",[],"['excel', 'powerpoint', 'qlik', 'spss', 'table...",[]
929,2314,"(US) Senior Research Scientist, Life Sciences",PointClickCare,NaN,https://www.linkedin.com/jobs/view/4245374099,NaN,2025-06-09,Remote,Full-time,"Reporting to the VP of Operations, Life Scienc...",...,NaN,138000.0,138000.0,138000.0,0.6,Senior,[],[],[],[]
930,2315,Junior Data Scientist,OGC Global,NaN,https://www.linkedin.com/jobs/view/4244767354,NaN,2025-06-08,Remote,Full-time,OGC is a boutique research and consulting agen...,...,NaN,NaN,NaN,NaN,NaN,Entry-Level,"['python', 'r', 'sql', 'vba']",[],"['excel', 'powerpoint', 'qlik', 'spss', 'table...",[]
943,2329,Senior Prompt & AI/ML Engineer - Remote,Uplers,NaN,https://www.linkedin.com/jobs/view/4232768128,NaN,2025-06-09,Remote,Full-time,**Experience** \r\n : 3\.00 \+ years\r\n \r\...,...,NaN,NaN,NaN,NaN,NaN,Senior,"['js', 'python', 'r']",[],"['azure', 'docker', 'gdpr', 'graphql', 'mysql']",['azure']
946,2339,Data Analyst (100% Remote),Lensa,NaN,https://www.linkedin.com/jobs/view/4248366670,NaN,NaN,Remote,Full-time,Lensa is the leading career site for job seeke...,...,NaN,81000.0,81000.0,81000.0,0.6,Not Specified,"['c', 'python', 'r', 'sql']",[],"['dax', 'excel', 'jira', 'sharepoint', 'tableau']",[]
949,2349,Data Analyst,Itaú Chile,NaN,https://www.linkedin.com/jobs/view/4248221181,NaN,NaN,Hybrid,Full-time,"**¿Eres de esas personas de mente inquieta, co...",...,NaN,NaN,NaN,NaN,NaN,Not Specified,['sql'],[],['tableau'],[]
962,2368,Machine Learning Engineer,Ascendion,NaN,https://www.linkedin.com/jobs/view/4245365033,linkedin,2025-06-09,Remote,Full-time,About Ascendion\r\n \r\n\r\n\r\n\r\n \r\n\r\n...,...,NaN,130000.0,130000.0,130000.0,0.6,Not Specified,['python'],[],"['azure', 'docker', 'pytorch', 'tensorflow']",['azure']
965,2375,Data Scientist (L5) - Member Algorithm Foundat...,Netflix,NaN,https://www.linkedin.com/jobs/view/4138391092,linkedin,NaN,Remote,Full-time,Netflix is one of the world's leading entertai...,...,NaN,170000.0,170000.0,170000.0,0.6,Not Specified,['python'],[],['excel'],[]
969,2384,Senior Python / AI Engineer : Hybrid (Mumbai),Uplers,NaN,https://www.linkedin.com/jobs/view/4231582347,linkedin,2025-06-07,Remote,Full-time,**Experience** \r\n : 8\.00 \+ years\r\n \r\...,...,NaN,48900.0,48900.0,48900.0,0.6,Senior,['python'],[],"['pytorch', 'tensorflow']",[]


In [37]:
# Populate missing 15 locations

df_silver.loc[df_silver['id'] == 2279, 'location'] = "New York, NY"
df_silver.loc[df_silver['id'] == 2304, 'location'] = "United States"
df_silver.loc[df_silver['id'] == 2314, 'location'] = "United States"
df_silver.loc[df_silver['id'] == 2315, 'location'] = "United States"
df_silver.loc[df_silver['id'] == 2329, 'location'] = "India"
df_silver.loc[df_silver['id'] == 2339, 'location'] = "United States"
df_silver.loc[df_silver['id'] == 2349, 'location'] = "Chile"
df_silver.loc[df_silver['id'] == 2368, 'location'] = "United States"
df_silver.loc[df_silver['id'] == 2375, 'location'] = "United States"
df_silver.loc[df_silver['id'] == 2384, 'location'] = "India"
df_silver.loc[df_silver['id'] == 2397, 'location'] = "India"
df_silver.loc[df_silver['id'] == 2448, 'location'] = "India"
df_silver.loc[df_silver['id'] == 2478, 'location'] = "United States"
df_silver.loc[df_silver['id'] == 2513, 'location'] = "United States"
df_silver.loc[df_silver['id'] == 2514, 'location'] = "United States"

## Fact checking salary-related columns

We need to verify the reliability of all extracted salary-related columns.

In [42]:
# List all salary related columns
salary_cols = [col for col in df_silver.columns if "salary" in col]
df_silver[salary_cols]

,has_salary,min_salary_raw,max_salary_raw,single_salary_raw,salary_period,min_salary_annual_usd,max_salary_annual_usd,avg_salary_annual_usd,salary_confidence
0,True,NaN,NaN,124000.0,NaN,124000.0,124000.0,124000.0,0.6
1,True,NaN,NaN,124000.0,NaN,124000.0,124000.0,124000.0,0.6
2,True,NaN,NaN,124000.0,NaN,124000.0,124000.0,124000.0,0.6
3,True,NaN,NaN,124000.0,NaN,124000.0,124000.0,124000.0,0.6
4,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1043,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1044,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1045,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1046,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Save current dataframe to overwrite `enriched_jobs.csv` to fix max_salary_annual_usd

We need to run `src/rerun_salary_extraction.py` which will extract salary ranges correctly. So save this modified dataframe for input to `src/rerun_salary_extraction.py`.

In [47]:
# Overwrite the original enriched_jobs.csv with the new dataframe
df_silver.to_csv("../data/silver/enriched_jobs.csv", index=False)

# Normalized `location` field

We created `src/extractors/location_extractor.py` as module for normalizing `location` field into a new `country` column:
Important library:
!pip install geopy

First, we removed obfuscated rows:

In [22]:
# Remove Obfuscated Rows from Data
import pandas as pd
from pathlib import Path
from src.extractors.obfuscation_cleaner import drop_obfuscated_rows
from src.extractors.location_extractor import extract_country
# Load your data
input_path = Path("../data/silver/enriched_jobs.csv")
df = pd.read_csv(input_path)

# Drop obfuscated rows (keep only rows with real data in any column except 'link')
df_clean = drop_obfuscated_rows(df, exclude_cols=['link'])


In [23]:
# Extract country for each location
df_clean['country'] = df_clean['location'].apply(extract_country)

# Overwrite the original file with the new country column
df_clean.to_csv(input_path, index=False)
print("Country extraction complete")

new york not found in regex
washington dc-baltimore not found in regex
new york not found in regex
greater bengaluru not found in regex
greater rio de janeiro not found in regex
greater minneapolis-st. paul not found in regex
san francisco bay not found in regex
lisbon not found in regex
washington dc-baltimore not found in regex
greater bengaluru not found in regex
lisbon not found in regex
porto not found in regex
san francisco bay not found in regex
greater chicago not found in regex
bangkok not found in regex
utica-rome not found in regex
greater kolkata not found in regex
greater istanbul not found in regex
dallas-fort worth not found in regex
salt lake not found in regex
columbus, ohio not found in regex
greater st. louis not found in regex
mumbai not found in regex
mumbai not found in regex
greater coventry not found in regex
new york not found in regex
new york not found in regex
san francisco bay not found in regex
washington dc-baltimore not found in regex
new york not found 

Country extraction complete
